In [3]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from sklearn.model_selection import train_test_split, GridSearchCV, \
                                    ShuffleSplit, KFold

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.preprocessing import MinMaxScaler, StandardScaler, \
                                  RobustScaler, PolynomialFeatures, \
                                  OrdinalEncoder, LabelEncoder, \
                                  OneHotEncoder, TargetEncoder, \
                                  QuantileTransformer, PowerTransformer, \
                                  KBinsDiscretizer

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, RANSACRegressor, \
                                 Ridge, Lasso, LinearRegression

from sklearn.metrics import recall_score, precision_score, \
                            f1_score, ConfusionMatrixDisplay, \
                            confusion_matrix, roc_auc_score, \
                            RocCurveDisplay, PrecisionRecallDisplay, \
                            roc_curve, precision_recall_curve, \
                            PrecisionRecallDisplay, make_scorer, \
                            mean_squared_error

from sklearn.compose import ColumnTransformer, make_column_selector,\
                            make_column_transformer, TransformedTargetRegressor
# import category_encoders as ce
from sklearn.metrics import fbeta_score
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_validate
from sklearn.ensemble import VotingClassifier
import pickle

In [5]:
path_test  = 'test.csv'
path_train = 'train.csv'
path_train_y = 'train_Y.csv'

In [6]:
data_test = pd.read_csv(path_test)

In [7]:
# Блок обучения модели (все три модели)

In [8]:
data_raw = pd.read_csv(path_train)
train_y = pd.read_csv(path_train_y)
data_all = pd.concat([data_raw, train_y], axis=1)
data_2 = data_all.drop_duplicates(keep='first', subset=['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income'])

In [9]:
model_pipeline_f = Pipeline(steps=[
    ('preprocessing', StandardScaler()),
    ('model', RandomForestClassifier(n_estimators=2000, random_state=13,
                               oob_score = True, n_jobs=-1, criterion='entropy'))])
display(model_pipeline_f)

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('model',
                 RandomForestClassifier(criterion='entropy', n_estimators=2000,
                                        n_jobs=-1, oob_score=True,
                                        random_state=13))])

In [10]:
forest_model = model_pipeline_f.fit(data_2.drop(columns='Diabetes_binary'), data_2['Diabetes_binary'])

In [11]:
with open('gbrt_model.pkl', 'rb') as f:
    gbrt_model = pickle.load(f)

In [12]:
pipeline_KNN = Pipeline(steps=[
    ('preprocessing', StandardScaler()),
    ('model', KNeighborsClassifier(n_neighbors=24, p=4))])

In [13]:
pipeline_svc = Pipeline(steps=[
    ('preprocessing', StandardScaler()),
    ('model', SVC(kernel='rbf', gamma=8, C=1000))])

In [14]:
clf_v1 = RandomForestClassifier(n_estimators=1000, random_state=13,
                               oob_score = True, n_jobs=-1, criterion='entropy')
clf_v2 = GradientBoostingClassifier(random_state=13, loss='exponential', max_depth=4, n_estimators=4000, learning_rate=0.01, subsample=0.75)
clf_v3 = pipeline_svc
clf_v4 = pipeline_KNN


e_s_clf = StackingClassifier(
        estimators = [
            ('RF', clf_v1), 
            ('GB', clf_v2), 
            ('SVC', clf_v3),
            ('KNN', clf_v4)],
        final_estimator = LogisticRegression(max_iter=100),
        stack_method = 'predict'
)

In [15]:
stack_model = e_s_clf.fit(data_2.drop(columns='Diabetes_binary'), data_2['Diabetes_binary'])

In [16]:
# Блок предсказания с использованием тестового набора (все три модели)

In [21]:
# Вектора предсказанных значений  RandomForestClassifier или ExtraTreesClassifier
y_predict_random_forest = forest_model.predict(data_test)

In [22]:
# Вектора предсказанных значений  GradientBoostingClassifier или HistGradientBoostingClassifier
y_predict_gradient_boosting = gbrt_model.predict(data_test)

In [23]:
# Вектора предсказанных значений  финальной модели
y_predict_final = stack_model.predict(data_test)

In [33]:
pd.Series(y_predict_random_forest).to_csv('y_pred_random_forest.csv', index=False)

In [34]:
pd.Series(y_predict_gradient_boosting).to_csv('y_pred_gradient_boosting.csv', index=False)

In [35]:
pd.Series(y_predict_final).to_csv('y_pred_final.csv', index=False)

In [38]:
display(pd.read_csv('y_pred_random_forest.csv'))

,0
0,1.0
1,1.0
2,0.0
3,1.0
4,1.0
...,...
23324,0.0
23325,1.0
23326,1.0
23327,1.0
